In [1]:
import cv2
import numpy as np
import pandas as pd
import os
import time
from google.colab import drive
drive.mount('/content/drive')
color_ranges = {
    'red': ((0, 100, 100), (10, 255, 255)),
    'green': ((50, 100, 100), (70, 255, 255)),
    'blue': ((100, 100, 100), (130, 255, 255))
}

def detect_balls(frame, color_ranges):
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    detected_balls = []

    for color, (lower, upper) in color_ranges.items():
        mask = cv2.inRange(hsv_frame, np.array(lower), np.array(upper))
        contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        for contour in contours:
            area = cv2.contourArea(contour)
            if area > 500:  # Filter by area
                x, y, w, h = cv2.boundingRect(contour)
                detected_balls.append((color, x + w//2, y + h//2))

    return detected_balls

def get_quadrant(x, y, width, height):
    if x < width // 2:
        if y < height // 2:
            return 1
        else:
            return 3
    else:
        if y < height // 2:
            return 2
        else:
            return 4

def process_video(video_path, output_video_path, output_text_path):
    os.makedirs(os.path.dirname(output_video_path), exist_ok=True)
    os.makedirs(os.path.dirname(output_text_path), exist_ok=True)
    cap = cv2.VideoCapture(video_path)
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    fps = cap.get(cv2.CAP_PROP_FPS)
    out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'XVID'), fps, (frame_width, frame_height))
    event_log = []
    ball_positions = {}
    start_time = time.time()
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        current_time = time.time() - start_time
        detected_balls = detect_balls(frame, color_ranges)

        for color, x, y in detected_balls:
            quadrant = get_quadrant(x, y, frame_width, frame_height)

            if color not in ball_positions:
                ball_positions[color] = quadrant
                event_log.append((current_time, quadrant, color, 'Entry'))
                cv2.putText(frame, f'{color} Entry at {current_time:.2f}s', (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
            elif ball_positions[color] != quadrant:
                event_log.append((current_time, quadrant, color, 'Entry'))
                event_log.append((current_time, ball_positions[color], color, 'Exit'))
                cv2.putText(frame, f'{color} Entry at {current_time:.2f}s', (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
                ball_positions[color] = quadrant

        out.write(frame)

    cap.release()
    out.release()
    with open(output_text_path, 'w') as f:
        for event in event_log:
            f.write(f'{event[0]:.2f}, {event[1]}, {event[2]}, {event[3]}\n')

input_video_path = '/content/drive/MyDrive/Anemia iron deficency (nails)/AI Assignment video.mp4'
output_video_path = '/content/drive/My Drive/output/processed_video.avi'
output_text_path = '/content/drive/My Drive/output/event_log.txt'

process_video(input_video_path, output_video_path, output_text_path)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
